In [73]:
%matplotlib inline

from music21 import pitch
import sqlite3
import sqlalchemy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pomegranate import *


In [4]:
name = 'wjazzd.db'
jazzdb = sqlite3.connect(name)
cursor = jazzdb.cursor()
cursor.execute("SELECT name FROM sqlite_master;")
tables = cursor.fetchall()
tables

[(u'melody',),
 (u'sqlite_sequence',),
 (u'beats',),
 (u'sections',),
 (u'melody_type',),
 (u'transcription_info',),
 (u'solo_info',),
 (u'record_info',),
 (u'track_info',),
 (u'composition_info',),
 (u'esac_info',),
 (u'popsong_info',),
 (u'db_info',),
 (u'melind',),
 (u'esacind',),
 (u'soloind',),
 (u'transind',),
 (u'trackind',),
 (u'compind',),
 (u'sectind',),
 (u'typeind',)]

In [195]:
engine = sqlalchemy.create_engine("sqlite:///%s" % name, execution_options={"sqlite_raw_colnames": True})
melody = pd.read_sql_table('melody', engine)
beats = pd.read_sql_table('beats', engine)
solo_info = pd.read_sql_table('solo_info', engine)
record_info = pd.read_sql_table('record_info', engine)
composition_info = pd.read_sql_table('composition_info', engine)

In [197]:
melody.loc[melody['melid'] == 5]
print 'melody columns'
print melody.columns.values

melody_f = melody.filter(['eventid', 'melid', 'pitch'])
#solo_info['key'].value_counts().plot(kind='bar')

melody columns
['eventid' 'melid' 'onset' 'pitch' 'duration' 'period' 'division' 'bar'
 'beat' 'tatum' 'subtatum' 'num' 'denom' 'beatprops' 'beatdur'
 'tatumprops' 'f0_mod' 'loud_max' 'loud_med' 'loud_sd' 'loud_relpos'
 'loud_cent' 'loud_s2b' 'f0_range' 'f0_freq_hz' 'f0_med_dev']


In [200]:
transpose_dest_midi = pitch.Pitch('B-').midi

# find the distance to destination key
# returns (dist, minor/maj) for usable songs or None for unusable songs
def get_key_transpose(s):
    try:
        n = s.split('-')[0]
        m = s.split('-')[1]
        if m not in ['maj', 'min']:
            return None
        if n[-1] == 'b':
            n = n[:-1] + '-'
        note_midi = pitch.Pitch(n).midi

        # move to relative major of minor key
        if m == 'min':
            mode = 'minor'
            note_midi += 3
        else:
            mode = 'major'
            
        # find smallest distance to the destination pitch
        dist = transpose_dest_midi - note_midi
        if dist > 6:
            dist -= 12
        elif dist < -6:
            dist += 12
        return dist, mode
    
    except:
        if s == '':
            return None
        elif s == 'Ab':
            return 2, 'major'
        else:
            return None

In [201]:
transposition_dists = {}
for i, r in solo_info.iterrows():
    transposition_dists[r['melid']] = get_key_transpose(r['key'])


In [167]:
# melody.apply(lambda x: transposition_dists.get(x['melid']) is not None)
# m2 = melody[melody.apply(lambda x: transposition_dists[x['melid']] is not None, axis=1)]
# melody.apply(lambda x: x['pitch'] += transposition_dists[x['melid']][0], axis=1)

SyntaxError: invalid syntax (<ipython-input-167-043151ad33a5>, line 3)

In [214]:
melody_filtered = melody.copy()
notes_by_melody = {}
for k, v in transposition_dists.iteritems():
    if v is None:
        melody_filtered = melody_filtered.loc[melody_filtered['melid'] != k]
    else:
        melody_filtered.loc[melody_filtered['melid'] == k, 'pitch'] += v[0]
        notes_by_melody[k] = melody_filtered.loc[melody_filtered['melid'] == k]
print 'filtered melody df, from %s to %s' % (melody.shape[0], melody_filtered.shape[0])

filtered melody df, from 200809 to 180597


In [215]:
for k, v in notes_by_melody.iteritems():
    notes_by_melody[k].filter(['eventid', 'melid', 'pitch'])

In [216]:
notes_by_melody[1]

,eventid,melid,onset,pitch,duration,period,division,bar,beat,tatum,...,f0_mod,loud_max,loud_med,loud_sd,loud_relpos,loud_cent,loud_s2b,f0_range,f0_freq_hz,f0_med_dev
0,1,1,10.343492,65.0,0.138776,4,1,0,1,1,...,,0.126209,66.526087,5.541147,0.307692,0.389466,1.056169,37.794261,12.932532,-0.328442
1,2,1,10.637642,63.0,0.171247,4,4,0,2,1,...,,0.349751,69.133321,2.912412,0.250000,0.468687,1.120317,6.365930,6.956935,11.135423
2,3,1,10.843719,58.0,0.081270,4,4,0,2,4,...,,0.094051,66.352130,3.564563,0.428571,0.531354,1.310389,68.010392,NaN,32.366787
3,4,1,10.948209,61.0,0.235102,4,1,0,3,1,...,,0.521187,66.484173,2.414298,0.818182,0.559333,0.984047,15.443906,5.867151,-3.374696
4,5,1,11.232653,63.0,0.130612,4,1,0,4,1,...,,0.560737,71.699054,2.185794,0.166667,0.438973,1.061262,11.444363,8.329975,6.377737
5,6,1,11.551927,58.0,0.188662,4,1,1,1,1,...,,0.534657,67.636708,7.635221,0.411765,0.359536,1.049956,39.368720,6.589582,16.146429
6,7,1,11.859592,58.0,0.481814,4,1,1,2,1,...,vibrato,0.584914,63.659343,5.518070,0.068182,0.403372,0.983151,39.429103,5.406750,11.239471
7,8,1,14.535692,50.0,0.159637,4,1,3,3,1,...,,-0.129185,58.507975,5.020340,0.133333,0.368384,0.927912,174.398513,NaN,25.203232
8,9,1,14.799819,57.0,0.145125,4,2,3,4,1,...,,0.599931,71.173670,2.938194,0.285714,0.551884,1.064195,27.066543,7.758283,25.736430
9,10,1,14.973968,60.0,0.110295,4,2,3,4,2,...,,0.484532,69.632891,2.325457,0.600000,0.508617,1.038483,17.141304,11.184763,15.693739
